In [1]:
from binary_param import linear, constant
import pandas as pd
import numpy as np
from R125_bmimBF4_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = 0.6
    init_kappa_1_2A = 0.028
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimBF4'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimBF4','bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_B['bmimBF4', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'bmimBF4'].fix(0)
    m.fs.properties.PR_kappa_C['bmimBF4', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimBF4'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data['x_bmimBF4']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimBF4'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimBF4', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimBF4']"]

In [3]:
data = pd.read_csv('R125_bmimBF4.csv')
print(data)

   temperature  pressure  x_R125  x_bmimBF4
0       298.15     50000   0.008      0.992
1       298.15    100000   0.019      0.981
2       298.15    200000   0.043      0.957
3       298.15    400000   0.101      0.899
4       298.15    600000   0.172      0.828
5       298.15    799500   0.263      0.737
6       298.15   1000000   0.378      0.622


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()

    data_["x_bmimBF4"] += data_["x_bmimBF4"] * noise_x

    data_.loc[data_['x_bmimBF4'] <= 0.0001, 'x_bmimBF4'] = 0.0001
    data_.loc[data_['x_bmimBF4'] >= 0.9999 , 'x_bmimBF4'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.027813126306829458
2022-03-06 00:22:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:22:34 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
    model.name="_Q_opt";
      - termination condition: maxIterations
      - mess

2022-03-06 00:24:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:35 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.47246077637430106
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.056641316525731594
8
-0.15968055820697585
2022-03-06 00:24:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:24:43 [INFO] idaes.init.fs.state_block: Property package initializ

fs.properties.PR_kappa_A[bmimBF4,R125] = 0.5418708356193426
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.06222702886051885
15
-0.13883995764948132
2022-03-06 00:26:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:26:36 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeas

fs.properties.PR_kappa_A[bmimBF4,R125] = 0.8180891314679304
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.07860393447270748
22
0.03657884815502934
2022-03-06 00:28:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:28:17 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasi

2022-03-06 00:29:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:29:52 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.8019139344339806
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.0764213128265943
29
0.02665438104143176
2022-03-06 00:29:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:29:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:29:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:2

...

    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimBF4,R125] = 1.2493017829723891
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.1121668062043487
72
-0.043027344428929894
2022-03-06 00:39:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:39:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:39:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:39:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:39:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:39:57 [INFO] idaes.init.fs.state_block: Propert

2022-03-06 00:41:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:41:15 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.3847597801535272
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.051935196826853845
79
0.1100248485209507
2022-03-06 00:41:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:41:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:41:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:41:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:41:23 [INFO] idaes.init.fs.state_block: Property package initializat

2022-03-06 00:42:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:42:43 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.8263690289789761
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.07801093666908454
86
-0.1903930317408165
2022-03-06 00:42:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:42:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:42:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:

2022-03-06 00:44:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:44:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:44:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:44:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:44:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:44:16 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
fs.properties.PR_kappa_A[bmimBF4,R125] = 0.3645921187283463
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.050515559744586304
93
0.022903547252213004
2022-03-06 00:44:22 [INFO] idaes.init.fs.state_block: Property package initializ

2022-03-06 00:45:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:45:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:45:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 00:45:45 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
    model.name="_Q_opt";
      - termination condition: maxIterations
      - message from solver: Ipopt 3.13.2\x3a Maximum Number of Iterations
        Exceeded.
fs.properties.PR_kappa_A[bmimBF4,R125] = 1.7056810346170102
fs.properties.PR_kappa_A[R125,bmimBF4] = 0.10755898522484289


In [5]:
print(results_params)

[[1.25661025 0.09590818]
 [0.55591116 0.06332473]
 [0.0526903  0.02204431]
 [       nan        nan]
 [0.83037457 0.08042   ]
 [0.42181438 0.05236662]
 [0.49143338 0.0581968 ]
 [0.47246078 0.05664132]
 [0.42821453 0.05291709]
 [0.76088135 0.0689955 ]
 [0.81633807 0.07610662]
 [0.35452603 0.04979913]
 [1.38949011 0.10672324]
 [0.43888155 0.05564013]
 [0.54187084 0.06222703]
 [0.20874036 0.03878897]
 [0.51890907 0.06041001]
 [0.49451986 0.05844768]
 [0.71328494 0.0668695 ]
 [0.80787387 0.06719876]
 [1.27388357 0.11113388]
 [0.81808913 0.07860393]
 [0.84473568 0.08249053]
 [0.7757625  0.07333597]
 [0.5608786  0.06371082]
 [1.64491682 0.10648341]
 [1.27365602 0.11269061]
 [0.78609419 0.07344738]
 [0.80191393 0.07642131]
 [0.81500389 0.0782161 ]
 [0.45126786 0.05645732]
 [0.32073525 0.04735496]
 [0.30425502 0.04367744]
 [1.52716948 0.10235983]
 [1.27543607 0.11382877]
 [0.52500209 0.06089493]
 [0.54710531 0.06263741]
 [0.76462753 0.06837266]
 [0.52061981 0.06054637]
 [0.76413609 0.06636307]


In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmimbf4_10_2.csv')